In [168]:
import numpy as np
import pandas as pd

train_data = pd.read_csv('./train.csv')
test_data = pd.read_csv('./test.csv')

train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [169]:
test_data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [170]:
test_data.isnull().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

In [171]:
women = train_data.loc[train_data.Sex == 'female']["Survived"]
rate_women = sum(women)/len(women)

print("% of women who survived:", rate_women)

% of women who survived: 0.7420382165605095


In [172]:
men = train_data.loc[train_data.Sex == 'male']["Survived"]
rate_men = sum(men)/len(men)

print("% of men who survived:", rate_men)

% of men who survived: 0.18890814558058924


In [173]:
from sklearn.ensemble import RandomForestClassifier

y = train_data["Survived"]

features = ["Pclass", "Sex", "SibSp", "Parch"] #Score 0.77511
X = pd.get_dummies(train_data[features])
X_test = pd.get_dummies(test_data[features])

model = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=1) #depth of 30 makes Score 0.77272
model.fit(X, y)
Y_predict = model.predict(X_test)

output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': Y_predict})
output.to_csv('submission.csv', index=False)
print("Your submission was successfully saved!")

Your submission was successfully saved!


In [174]:
X.head()

,Pclass,SibSp,Parch,Sex_female,Sex_male
0,3,1,0,False,True
1,1,1,0,True,False
2,3,0,0,True,False
3,1,1,0,True,False
4,3,0,0,False,True


In [175]:
train_data.dtypes

PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object

In [176]:
len(output)

418

In [179]:
singleList = []
totalFamily =[]
for index, row in train_data.iterrows():
    totalFamily.append(row['SibSp']+row['Parch'])
    name = row['Name']
    if ("Miss." in name) or ("Master." in name):
        #print(name + " Single")
        singleList.append(1)
    elif ("Mr." in name) or ("Mrs." in name):
        singleList.append(0)
    else:
        #print(name + " Married")
        singleList.append(1)

singleList2 = []
totalFamily2 =[]
for index, row in test_data.iterrows():
    totalFamily2.append(row['SibSp']+row['Parch'])
    name = row['Name']
    if ("Miss." in name) or ("Master." in name):
        #print(name + " Single")
        singleList2.append(1)
    elif ("Mr." in name) or ("Mrs." in name):
        singleList2.append(0)
    else:
        #print(name + " Married")
        singleList2.append(1)

In [180]:
print(len(singleList))
train_data.insert(12, "Single", singleList)
train_data.insert(13, "totalFam", totalFamily)
train_data.head()
print(len(singleList2))
test_data.insert(11, "Single", singleList2)
test_data.insert(12, "totalFam", totalFamily2)
test_data.head()

891
418


,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Single,totalFam
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q,0,0
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S,0,1
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q,0,0
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S,0,0
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S,0,2


In [181]:
from sklearn.ensemble import RandomForestClassifier

y = train_data["Survived"]

#features = ["Pclass", "Sex", "SibSp", "Parch", "Single"]  #Score is 0.76555
#features = ["Pclass", "Sex", "Single"] #Score is 0.78468
#features = ["Pclass", "Sex", "Parch", "Single"] #Score is 0.77033
#features = ["Pclass", "Sex","totalFam", "Single"] #Score is 0.77511
features = ["Sex","totalFam", "Single"] #Score is 0.77033
X = pd.get_dummies(train_data[features])
X_test = pd.get_dummies(test_data[features])

model = RandomForestClassifier(n_estimators=200, max_depth=30, random_state=1)
model.fit(X, y)
Y_predict = model.predict(X_test)

output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': Y_predict})
output.to_csv('submissionRF.csv', index=False)
print("Your submissionRF was successfully saved!")

Your submissionRF was successfully saved!


In [182]:
from sklearn import svm

y = train_data["Survived"]

#features = ["Pclass", "Sex", "SibSp", "Parch", "Single"] 
#features = ["Pclass", "Sex", "Single"] #Score is 0.77511
features = ["Pclass", "Sex","totalFam", "Single"] #Score is 0.77751
X = pd.get_dummies(train_data[features])
X_test = pd.get_dummies(test_data[features])

model = svm.SVC(max_iter=-1, random_state=1)
model.fit(X, y)
Y_predict = model.predict(X_test)

output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': Y_predict})
output.to_csv('submissionSVM.csv', index=False)
print("Your submissionSVM was successfully saved!")

Your submissionSVM was successfully saved!


In [188]:
from sklearn import svm
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

y = train_data["Survived"]

#features = ["Pclass", "Sex", "SibSp", "Parch", "Single"] 
#features = ["Pclass", "Sex", "Single"] #Score is 0.77511
features = ["Pclass", "Sex","totalFam", "Single"] #Score is 0.77751

X = pd.get_dummies(train_data[features])
X_test = pd.get_dummies(test_data[features])

pca = PCA(n_components=3) # 3 has score of 0.78468 with "Pclass", "Sex","totalFam", "Single" and no scaler

X_train = pca.fit_transform(X)
X_test = pca.transform(X_test)

model = svm.SVC(max_iter=-1, random_state=1)
model.fit(X_train, y)
Y_predict = model.predict(X_test)

output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': Y_predict})
output.to_csv('submissionPCASVM.csv', index=False)
print("Your submissionPCASVM was successfully saved!")

Your submissionPCASVM was successfully saved!


In [189]:
from sklearn.linear_model import LogisticRegression

y = train_data["Survived"]

#features = ["Pclass", "Sex", "SibSp", "Parch"] #Score 0.77511
features = ["Pclass", "Sex","totalFam", "Single"]
X = pd.get_dummies(train_data[features])
X_test = pd.get_dummies(test_data[features])

model = LogisticRegression() 
model.fit(X, y)
Y_predict = model.predict(X_test)

output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': Y_predict})
output.to_csv('submissionLR.csv', index=False)
print("Your submissionLR was successfully saved!")

Your submissionLR was successfully saved!
